In [1]:
%micropython --info

{'ver': 'v1.21.0',
 'version': '1.21.0',
 'port': 'rp2',
 'mpy': 'v6.1',
 'build': '',
 'cpu': 'RP2040',
 'family': 'micropython',
 'board': 'Raspberry Pi Pico W with RP2040',
 'arch': 'armv6m',
 'serial_port': 'auto'}

In [3]:
from typing_extensions import TYPE_CHECKING  # type: ignore

if TYPE_CHECKING:
    from rp2.asm_pio import *

In [4]:
# %%micropython
from rp2 import PIO

# remove all programs from both PIOs
for n in (0, 1):
    pio = PIO(n)
    pio.remove_program()

In [5]:
# %%micropython
# Example using PIO to create a UART TX interface

from machine import Pin
from rp2 import PIO, StateMachine, asm_pio

UART_BAUD = 115200
PIN_BASE = 10
NUM_UARTS = 8


@asm_pio(
    sideset_init=PIO.OUT_HIGH,
    out_init=PIO.OUT_HIGH,
    out_shiftdir=PIO.SHIFT_RIGHT,
)
# fmt: off
def uart_tx():
    # Block with TX deasserted until data available
    pull()
    # Initialise bit counter, assert start bit for 8 cycles
    set(x, 7)  .side(0)       [7]
    # Shift out 8 data bits, 8 execution cycles per bit
    label("bitloop")
    out(pins, 1)              [6]
    jmp(x_dec, "bitloop")
    # Assert stop bit for 8 cycles total (incl 1 for pull())
    nop()      .side(1)       [6]
# fmt: on

i = 0
sm = StateMachine(
    7, uart_tx, freq=8 * UART_BAUD, sideset_base=Pin(PIN_BASE + i), out_base=Pin(PIN_BASE + i)
)
sm.active(1)


# We can print characters from each UART by pushing them to the TX FIFO
def pio_uart_print(sm, s):
    # for c in s:
    #     sm.put(ord(c))
    sm.put(s)


# import time
# while 1:
#     print("en weer")
#     pio_uart_print(sm, "Hello from UART 0")
#     time.sleep(1)

In [15]:
# start capture
# setup logic capture 
from saleae import automation
manager = automation.Manager.connect()
# setup logic capture on all channels
device_configuration = automation.LogicDeviceConfiguration(
    enabled_digital_channels=[0,1,2,3,4,5,6,7],
    digital_sample_rate=12_000_000,
)
capture = manager.start_capture(device_configuration=device_configuration)

In [16]:
# %%micropython
import time



for i in range(10):
    # print("en weer")
    pio_uart_print(sm, f"Hello {i} from UART {str(sm)}\n")
    time.sleep(0.1)

In [17]:
# End capture
capture.stop()
# "Input Channel", "Bit Rate (Bits/s)", "Bits per Frame", "Stop Bits", "Parity Bit", "Significant Bit", "Signal inversion", "Mode"
capture.add_analyzer("Async Serial", settings={"Input Channel": 6, "Bit Rate (Bits/s)": 115200})
capture.add_analyzer("Async Serial", settings={"Input Channel": 1, "Bit Rate (Bits/s)": 115200})

AnalyzerHandle(analyzer_id=10093)